In [1]:
from os.path import join
from collections import Counter, defaultdict
import pandas as pd

DATA_DIR = './Data'

In [ ]:
predictions = pd.read_csv(join('./log/bert_08150347/', 'test_results.tsv'),
                          header=None, sep='\t')
predictions = predictions.idxmax(axis=1)

print(Counter(predictions))

In [ ]:
with open(join(DATA_DIR, 'subtask2_data_from_test.txt'), 'r') as f_data:
    data = []
    for line in f_data:
        data.append(line.strip('\n').split('\t'))

In [ ]:
def add_FM_LS(fm_ls_l, content):
    ret, replaced = [], False
    
    for fm_ls in fm_ls_l:
        if fm_ls[1] == content[1] and fm_ls[2] == content[2]:
            ret.append(fm_ls) if fm_ls[4] < content[4] else ret.append(content)
            replaced = True
        else:
            ret.append(fm_ls)
            
    if not replaced:
        ret.append(content)

    return ret

In [ ]:
classes = ["OTHERS", "FM_OBS", "FM_OBS_N", "FM_LS"]
results = defaultdict(dict)

re_count = 0
for i, line in enumerate(data):
    if predictions[i] == 0:
        # prediction is not "OTHERS"
        continue
        
    re_count += 1
    doc = line[0][:13]

    # get family member properties
    sof = "NA"
    if "@MaternalRelative$" in line[1]:
        sof = "Maternal"
    elif "@PaternalRelative$" in line[1]:
        sof = "Paternal"

    content = ""            
    if "@Status$" in line[1]:
        results[doc]["FM-LS"] = add_FM_LS(results.get(doc, {}).get("FM-LS", []),
                             (doc, line[-2].capitalize(), sof, "LivingStatus", line[-1][-1]))
    elif "@Disease$" in line[1]:
        if predictions[i] == 2:
            negated = "Negated"
        else:
            negated = "Non_Negated"
        results[doc]["FM-OBS"] = results.get(doc, {}).get("FM-OBS", []) + \
                                    [(doc, line[-2].capitalize(),  # family member
                                      sof, "Observation", line[-1].replace('_', ' '),
                                      negated)]

with open(join(DATA_DIR, 're_test_result.tsv'), 'w') as f_predict:
    for doc, contents in results.items():
        print(contents)
        for content in set(contents.get("FM-LS", [])):
            f_predict.write("\t".join(list(content)))
            f_predict.write("\n")
        for content in set(contents.get("FM-OBS", [])):
            f_predict.write("\t".join(list(content)))
            f_predict.write("\n")

In [ ]:
re_count